In [1]:
path = '../RMG-models/PCI2017/022-Thion/thermo.txt'
f = open(path, "r")
mech_lines = f.readlines()

In [2]:
import re

entries = []; # entries we care about: contain species name and atomic composition
for line in mech_lines[9:-1]:
    if re.match('^\s', line): continue
    if line[0] == '-': continue
    if line[0] == '!': continue
    entries.append(line)
entries

['n2                      n   2   00   00   0G   300.00   5000.00  1000.00      1\r\n',
 'he                      he  1   00   00   0G   300.00   5000.00  1000.00      1\r\n',
 'h2                      h   2   00   00   0G   300.00   5000.00  1000.00      1\r\n',
 'h                       h   1   00   00   0G   300.00   5000.00  1000.00      1\r\n',
 'o                       o   1   00   00   0G   300.00   5000.00  1000.00      1\r\n',
 'oh                S 9/01o   1h   1    0    0G   200.000  6000.000 1000.        1\r\n',
 'ho2               T 1/09h  1.o  2.   0.   0.G   200.000  5000.000 1000.        1\r\n',
 'o2                      o   2   00   00   0G   300.00   5000.00  1000.00      1\r\n',
 'h2o               L 5/89h   2o   1    0    0G   200.000  6000.000 1000.        1\r\n',
 'h2o2              T 8/03h  2.o  2.   0.   0.G   200.000  6000.000 1000.        1\r\n',
 'co                RUS 79c   1o   1    0    0G   200.000  6000.000 1000.        1\r\n',
 'co2               L 7/88c

In [3]:
# dictionary of dictionaries for entries:
    # 1st value is name of species
    # 1st key is dictionary for atomic composition
        # 2nd value is element
        # 2nd key is number of atoms
entries_dict = {}
potential_errors = {}

for entry in entries:
    name = re.match('([^\s]+)\s', entry).group(1)
    not_name = entry[24:].strip()
    print name
    print not_name
    
    elements = []
    num_atoms = []
    for i in range(len(not_name.split())):
        string = not_name.split()[i]
        if re.search('[GLgl]', string) or len(string) > 3:
            break
        m = re.search("[a-zA-z]+", string) # find elements
        if not m:
            continue
        elements.append(m.group())
        m = re.search("\d+", not_name.split()[i+1]) # find corresponding number of atoms
        num = int(m.group())
        num_atoms.append(num)
        
        # if more than 10 atoms of same element in molecule, check for errors:
        if num >= 10:
            guess = 0
            # guess how many atoms should be present from name:
            for s in re.findall(elements[-1]+'(\d*)', name):
                if s: guess += int(s)
                else: guess += 1
            if num > guess:
                print "!!! POTENTIAL ERROR: too many {}".format(elements[-1])
                potential_errors[name] = dict(zip(elements, num_atoms))
            
    entries_dict[name] = dict(zip(elements, num_atoms))
    print entries_dict[name]
    print ""

n2
n   2   00   00   0G   300.00   5000.00  1000.00      1
{'n': 2}

he
he  1   00   00   0G   300.00   5000.00  1000.00      1
{'he': 1}

h2
h   2   00   00   0G   300.00   5000.00  1000.00      1
{'h': 2}

h
h   1   00   00   0G   300.00   5000.00  1000.00      1
{'h': 1}

o
o   1   00   00   0G   300.00   5000.00  1000.00      1
{'o': 1}

oh
o   1h   1    0    0G   200.000  6000.000 1000.        1
{'h': 1, 'o': 1}

ho2
h  1.o  2.   0.   0.G   200.000  5000.000 1000.        1
{'h': 1, 'o': 2}

o2
o   2   00   00   0G   300.00   5000.00  1000.00      1
{'o': 2}

h2o
h   2o   1    0    0G   200.000  6000.000 1000.        1
{'h': 2, 'o': 1}

h2o2
h  2.o  2.   0.   0.G   200.000  6000.000 1000.        1
{'h': 2, 'o': 2}

co
c   1o   1    0    0G   200.000  6000.000 1000.        1
{'c': 1, 'o': 1}

co2
c   1o   2    0    0G   200.000  6000.000 1000.        1
{'c': 1, 'o': 2}

ch2o
h  2.c  1.o  1.   0.G   200.000  6000.000 1000.        1
{'h': 2, 'c': 1, 'o': 1}

hco
c  1.h  1.o  1.   0.G 

c   3h   6o   3    0g   300.000  5000.000 1388.000    41
{'h': 6, 'c': 3, 'o': 3}

c3ket13
c   3h   6o   3    0g   300.000  5000.000 1381.000    41
{'h': 6, 'c': 3, 'o': 3}

c3ket21
c   3h   6o   3    0g   300.000  5000.000 1371.000    41
{'h': 6, 'c': 3, 'o': 3}

c3h6o1-3
c   3h   6o   1    0g   300.000  5000.000 2035.000    41
{'h': 6, 'c': 3, 'o': 1}

c3h6o1-2
c   3h   6o   1    0G   200.000  6000.000 1000.        1
{'h': 6, 'c': 3, 'o': 1}

ch3chco
c  3.h  4.o  1.   0.g   200.000  6000.000 1000.000     1
{'h': 4, 'c': 3, 'o': 1}

ch2cco
c   3h   2o   1    0g   300.000  5000.000 1000.000     1
{'h': 2, 'c': 3, 'o': 1}

hccco
c   3h   1o   1   0G   300.00   4000.00  1000.00      1
{'h': 1, 'c': 3, 'o': 1}

ch2cch2oh
c   3h   5o   1     G   300.000  5000.000 1000.00      1
{'h': 5, 'c': 3, 'o': 1}

c3h62oh
c   3h   7o   1   0G   300.00   3000.00  1000.00      1
{'h': 7, 'c': 3, 'o': 1}

c3h61oh
c   3h   7o   1   0G   300.00   3000.00  1000.00      1
{'h': 7, 'c': 3, 'o': 1}

c3h61oh2o

In [4]:
entries_dict

{'ac3h5cho': {'c': 4, 'h': 6, 'o': 1},
 'ac3h5co': {'c': 4, 'h': 5, 'o': 1},
 'but1ooh2*o': {'c': 4, 'h': 8, 'o': 3},
 'but1ooh2o2': {'c': 4, 'h': 9, 'o': 4},
 'but1ooh3o2': {'c': 4, 'h': 9, 'o': 4},
 'but1ooh4o2': {'c': 4, 'h': 9, 'o': 4},
 'but2ooh1*o': {'c': 4, 'h': 8, 'o': 3},
 'but2ooh1o2': {'c': 4, 'h': 9, 'o': 4},
 'but2ooh3o2': {'c': 4, 'h': 9, 'o': 4},
 'but2ooh4o2': {'c': 4, 'h': 9, 'o': 4},
 'but3ooh1*o': {'c': 4, 'h': 8, 'o': 3},
 'but3ooh2*o': {'c': 4, 'h': 8, 'o': 3},
 'but4ooh1*o': {'c': 4, 'h': 8, 'o': 3},
 'but4ooh2*o': {'c': 4, 'h': 8, 'o': 3},
 'butyn2': {'c': 4, 'h': 6},
 'butyne': {'c': 4, 'h': 6},
 'c': {'c': 1},
 'c2': {'c': 2},
 'c2c4h8': {'c': 4, 'h': 8},
 'c2h': {'c': 2, 'h': 1},
 'c2h2': {'c': 2, 'h': 2},
 'c2h2oh': {'N': 0, 'c': 2, 'h': 3, 'o': 1},
 'c2h3': {'c': 2, 'h': 3},
 'c2h3chcho': {'c': 4, 'h': 5, 'o': 1},
 'c2h3cho': {'c': 3, 'h': 4, 'o': 1},
 'c2h3choch2': {'c': 4, 'h': 6, 'o': 1},
 'c2h3co': {'c': 3, 'h': 3, 'o': 1},
 'c2h3coch2': {'c': 4, 'h': 5,

In [5]:
potential_errors

{}